In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import os
os.chdir("../")

In [2]:
load_dotenv(override=True)

True

In [3]:
from ficast.assembly.ficast import FiCast
from ficast.dialogue.speech import DialogueSynthesis
dialoguer = DialogueSynthesis(
  client_type="api", 
  base_url=os.getenv("TTS_API_BASE_URL"),
  api_key=os.getenv("TTS_API_KEY")
)
http_client = dialoguer.client.client
http_client

Loading config from path: /home/mymm_psu_gmail_com/FiCast-GPT
Loading config version: default from path: ../conf/dialogue


USER_AGENT environment variable not set, consider setting it to identify your requests.


Loading config version: default from path: ../conf/dialogue
Token verification successful.


In [29]:
dialoguer.client.all_voices

[Voice(voice_id='0', name='darth_vader', samples=None, category=None, fine_tuning=None, labels={'gender': 'unknown'}, description=None, preview_url=None, available_for_tiers=None, settings=None, sharing=None, high_quality_base_model_ids=None, safety_control=None, voice_verification=None, owner_id=None, permission_on_resource=None, is_legacy=None),
 Voice(voice_id='1', name='pat2', samples=None, category=None, fine_tuning=None, labels={'gender': 'unknown'}, description=None, preview_url=None, available_for_tiers=None, settings=None, sharing=None, high_quality_base_model_ids=None, safety_control=None, voice_verification=None, owner_id=None, permission_on_resource=None, is_legacy=None),
 Voice(voice_id='2', name='train_kennard', samples=None, category=None, fine_tuning=None, labels={'gender': 'unknown'}, description=None, preview_url=None, available_for_tiers=None, settings=None, sharing=None, high_quality_base_model_ids=None, safety_control=None, voice_verification=None, owner_id=None, p

In [75]:
# create task
character = "darth_vader"
payload = {
    "text": f"Hello, testing quickstart notebook demo by {character}.",
    "voice": character,
    "preset": ["fast", "standard"][0]
}
task_response = http_client.post(
    f"/tts", json=payload
)
task_response.raise_for_status()
if "task_id" in task_response.json():
    task_id = task_response.json()["task_id"]
    print(task_id)

68488c7e-46f7-4e6a-859b-df7c6025372c


In [76]:
# check status
status_response = http_client.get(
    f"/task/status/{task_id}", 
    timeout=None
)
status_response.raise_for_status()
print(status_response.json())

{'task_id': '68488c7e-46f7-4e6a-859b-df7c6025372c', 'status': 'PENDING'}


In [77]:
# get results
result_response = http_client.get(
    f"/task/result/{task_id}", 
    timeout=None
)
result_response.raise_for_status()

<Response [200 OK]>

In [64]:
# inspect result
myaudio_gen = result_response.iter_bytes()
for audio in myaudio_gen:
  print(len(audio))

178254


In [24]:
# `text_to_speech` use the above pipeline
test_audio_gen = dialoguer.client.text_to_speech(
  "Hello, testing quickstart notebook demo.", 
  voice="random",
  preset="ultra_fast"
)

Save Results

In [78]:
from ficast.dialogue.utils import collect_audio, save_bytes_to_wav

In [79]:
import datetime
test_audio = collect_audio(result_response.iter_bytes())
save_bytes_to_wav(test_audio, f"notebooks/samples/dialogue/tts-nb_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.wav")

In [33]:
import datetime
test_audio = collect_audio(test_audio_gen)
save_bytes_to_wav(test_audio, f'notebooks/samples/dialogue/tts-test-nb_{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}.wav')